In [1]:
import pandas as pd
import numpy as np

import folium # map rendering librar
from geopy.geocoders import Nominatim
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Upload the dataset saved in the previous exercise

In [2]:
df_toronto = pd.read_csv("Toronto_2.csv", index_col=0)
df_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8113,-79.1930
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389
...,...,...,...,...,...
98,M9N,York,Weston,43.7068,-79.5170
99,M9P,Etobicoke,Westmount,43.6949,-79.5323
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.6898,-79.5582
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.7432,-79.5876


Name of the column is wrong, change it

In [3]:
df_toronto.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

In [4]:
df_toronto

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8113,-79.1930
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389
...,...,...,...,...,...
98,M9N,York,Weston,43.7068,-79.5170
99,M9P,Etobicoke,Westmount,43.6949,-79.5323
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.6898,-79.5582
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.7432,-79.5876


In [5]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighbourhoods.


Getting the Toronto's coordinates to center the map 

In [6]:
address = 'Toronto, Ontario, CN'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6425012, -79.3871153.


In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [8]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], 
                                           df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

To restrict the data to analize, we will work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. 

In [9]:
df_data = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_data.head(40)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6784,-79.2941
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.6693,-79.3155
3,M4M,East Toronto,Studio District,43.6561,-79.3406
4,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935
5,M4P,Central Toronto,Davisville North,43.7135,-79.3887
6,M4R,Central Toronto,North Toronto West,43.7143,-79.4065
7,M4S,Central Toronto,Davisville,43.7020,-79.3853
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.6899,-79.3853
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.6861,-79.4025


centering the map in East Toronto

In [10]:
address = 'East Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East Toronto are 43.653963, -79.387207.


In [11]:
# create map of East Toronto using latitude and longitude values
map_etoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_data['Latitude'], df_data['Longitude'], df_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etoronto)  
    
map_etoronto

 Business Reply Mail Processing Centre 969 Eastern is far from the other Neighbourhood, so let's drop it.

In [12]:
df_data = df_data[df_data['Neighborhood'] != 'Business Reply Mail Processing Centre 969 Eastern']
df_data

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6784,-79.2941
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.6693,-79.3155
3,M4M,East Toronto,Studio District,43.6561,-79.3406
4,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935
5,M4P,Central Toronto,Davisville North,43.7135,-79.3887
6,M4R,Central Toronto,North Toronto West,43.7143,-79.4065
7,M4S,Central Toronto,Davisville,43.7020,-79.3853
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.6899,-79.3853
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.6861,-79.4025


Let's create the map again

In [13]:
# create map of East Toronto using latitude and longitude values
map_etoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_data['Latitude'], df_data['Longitude'], df_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etoronto)  
    
map_etoronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [14]:
CLIENT_ID = 'EYTA5NXDL1UUYLI1TXNWTP5NZTQHT32V22UXFJ5TZ5HIKZGG' # your Foursquare ID
CLIENT_SECRET = 'YZUHQIXNXAK3H0L3QXPSPDUJ11QF0RI3ROXK112ABODLUBIX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EYTA5NXDL1UUYLI1TXNWTP5NZTQHT32V22UXFJ5TZ5HIKZGG
CLIENT_SECRET:YZUHQIXNXAK3H0L3QXPSPDUJ11QF0RI3ROXK112ABODLUBIX


### test with the first neighborhood

In [15]:
neighborhood_latitude = df_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.6784, -79.2941.


Let's get the top 100 venues that are in The Beaches within a radius of 500 meters

In [16]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EYTA5NXDL1UUYLI1TXNWTP5NZTQHT32V22UXFJ5TZ5HIKZGG&client_secret=YZUHQIXNXAK3H0L3QXPSPDUJ11QF0RI3ROXK112ABODLUBIX&v=20180605&ll=43.6784,-79.2941&radius=500&limit=100'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e5fea1bb4b684001bd2e51b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.682900004500006,
    'lng': -79.28788950308227},
   'sw': {'lat': 43.6738999955, 'lng': -79.30031049691773}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 

All the information is in the items key. Let's create the get_category_type function 

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/mateus/anaconda3/envs/tensorflow_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Beaches Bake Shop,Bakery,43.680363,-79.289692
3,The Beech Tree,Gastropub,43.680493,-79.288846
4,Grover Pub and Grub,Pub,43.679181,-79.297215


Printing how many venues were returned by Foursquare

In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


## Explore Neighborhoods

#### Let's create a function to repeat the same process to all the neighborhoods 

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *etoronto_venues*

In [22]:
etoronto_venues = getNearbyVenues(names=df_data['Neighborhood'],
                                   latitudes=df_data['Latitude'],
                                   longitudes=df_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

Let's print the size and the head of the resulting df

In [23]:
print(etoronto_venues.shape)
etoronto_venues.head()

(1632, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.6784,-79.2941,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.6784,-79.2941,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.6784,-79.2941,Beaches Bake Shop,43.680363,-79.289692,Bakery
3,The Beaches,43.6784,-79.2941,The Beech Tree,43.680493,-79.288846,Gastropub
4,The Beaches,43.6784,-79.2941,Grover Pub and Grub,43.679181,-79.297215,Pub


Let's check how many venues were returned for each neighborhood

In [24]:
etoronto_venues.groupby('Neighborhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,92,92,92,92,92,92
"Brockton, Exhibition Place, Parkdale Village",39,39,39,39,39,39
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",56,56,56,56,56,56
"Cabbagetown, St. James Town",48,48,48,48,48,48
Central Bay Street,89,89,89,89,89,89
"Chinatown, Grange Park, Kensington Market",84,84,84,84,84,84
Christie,12,12,12,12,12,12
Church and Wellesley,88,88,88,88,88,88


Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(etoronto_venues['Venue Category'].unique())))

There are 220 uniques categories.


## Analyze Each Neighborhood

One hot encoding of the categories

In [26]:
# one hot encoding
etoronto_onehot = pd.get_dummies(etoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
etoronto_onehot['Neighborhood'] = etoronto_venues['Neighborhood'] 

# move neighbourhood column to the first column
fixed_columns = [etoronto_onehot.columns[-1]] + list(etoronto_onehot.columns[:-1])
etoronto_onehot = etoronto_onehot[fixed_columns]

etoronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
etoronto_onehot.shape

(1632, 220)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
etoronto_grouped = etoronto_onehot.groupby('Neighborhood').mean().reset_index()
etoronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.030000,0.010000,0.000000,0.020000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.010870,0.021739,0.000000,0.000000,0.000000,0.010870,...,0.000000,0.000000,0.000000,0.00,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.025641,0.000000,0.000000,0.025641,0.025641,0.000000,0.000000,0.000000,...,0.051282,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Central Bay Street,0.000000,0.000000,0.000000,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011236,0.000000,0.00,0.000000,0.011236,0.011236,0.011236,0.000000,0.000000
6,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.023810,0.011905,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.035714,0.000000,0.047619,0.011905,0.000000,0.000000
7,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Church and Wellesley,0.011364,0.000000,0.011364,0.011364,0.000000,0.011364,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.011364,0.000000,0.000000,0.011364
9,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.030000,0.010000,0.000000,0.020000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000


In [29]:
etoronto_grouped.shape

(38, 220)

#### Let's print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in etoronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etoronto_grouped[etoronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.06
2           Restaurant  0.05
3           Steakhouse  0.03
4  Japanese Restaurant  0.03


----Berczy Park----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.05
2        Hotel  0.05
3   Restaurant  0.04
4       Bakery  0.04


----Brockton, Exhibition Place, Parkdale Village----
                 venue  freq
0                 Café  0.08
1          Coffee Shop  0.08
2       Breakfast Spot  0.05
3            Gift Shop  0.05
4  Japanese Restaurant  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.07
2                Café  0.05
3  Italian Restaurant  0.05
4                 Bar  0.05


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.08
1  Pizza Place  0.06
2       Bakery  0.06
3        

#### Let's put that into a *pandas* dataframe

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighbourhood.

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighborhood'] = etoronto_grouped['Neighborhood']

for ind in np.arange(etoronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etoronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,Bar,Steakhouse,Thai Restaurant,Hotel,Gastropub,Gym,American Restaurant
1,Berczy Park,Coffee Shop,Café,Hotel,Bakery,Restaurant,Japanese Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Cocktail Bar
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Gift Shop,Japanese Restaurant,Thrift / Vintage Store,Pet Store,Brewery,Sandwich Place,Restaurant
3,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Restaurant,Bar,Italian Restaurant,Café,French Restaurant,Gym / Fitness Center,Bakery,Park,Electronics Store
4,"Cabbagetown, St. James Town",Coffee Shop,Bakery,Pizza Place,Restaurant,Café,Pharmacy,Italian Restaurant,Park,Outdoor Sculpture,Ice Cream Shop


## Cluster Neighborhoods

Run *k*-means to cluster the neighbourhood into 5 clusters.

In [33]:
# set number of clusters
kclusters = 3

etoronto_grouped_clustering = etoronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

etoronto_merged = df_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
etoronto_merged = etoronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

etoronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.6784,-79.2941,1,Pub,Miscellaneous Shop,Health Food Store,Bakery,Trail,Gastropub,Coffee Shop,Distribution Center,Dog Run,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538,1,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Café,Grocery Store,Brewery,Bubble Tea Shop,Restaurant,Pub,Cocktail Bar
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.6693,-79.3155,1,Park,Sandwich Place,Pizza Place,Ice Cream Shop,Burrito Place,Brewery,Fast Food Restaurant,Fish & Chips Shop,Board Shop,Pub
3,M4M,East Toronto,Studio District,43.6561,-79.3406,1,Park,Coffee Shop,Performing Arts Venue,Diner,Garden Center,Baseball Field,Gym,Coworking Space,Cuban Restaurant,Distribution Center
4,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935,0,Park,Photography Studio,Lawyer,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Finally, let's visualize the resulting clusters

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etoronto_merged['Latitude'], etoronto_merged['Longitude'], etoronto_merged['Neighborhood'], etoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [36]:
etoronto_merged.loc[etoronto_merged['Cluster Labels'] == 0, etoronto_merged.columns[[1] + list(range(5, etoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Park,Photography Studio,Lawyer,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
6,Central Toronto,0,Gym Pool,Playground,Garden,Park,Cosmetics Shop,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
8,Central Toronto,0,Gym,Thai Restaurant,Tennis Court,Playground,Park,Grocery Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
10,Downtown Toronto,0,Park,Grocery Store,Playground,Candy Store,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
19,Downtown Toronto,0,Harbor / Marina,Café,Music Venue,Park,Grocery Store,Greek Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
23,Central Toronto,0,Park,Bus Line,Trail,Restaurant,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
30,Downtown Toronto,0,Café,Grocery Store,Park,Athletics & Sports,Candy Store,Playground,Baby Store,Coffee Shop,Dance Studio,Deli / Bodega
34,West Toronto,0,Park,Residential Building (Apartment / Condo),Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


This cluster have venue related with outside and gyms, with  2nd venues variated;

#### Cluster 2

In [37]:
etoronto_merged.loc[etoronto_merged['Cluster Labels'] == 1, etoronto_merged.columns[[1] + list(range(5, etoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Pub,Miscellaneous Shop,Health Food Store,Bakery,Trail,Gastropub,Coffee Shop,Distribution Center,Dog Run,Doner Restaurant
1,East Toronto,1,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Café,Grocery Store,Brewery,Bubble Tea Shop,Restaurant,Pub,Cocktail Bar
2,East Toronto,1,Park,Sandwich Place,Pizza Place,Ice Cream Shop,Burrito Place,Brewery,Fast Food Restaurant,Fish & Chips Shop,Board Shop,Pub
3,East Toronto,1,Park,Coffee Shop,Performing Arts Venue,Diner,Garden Center,Baseball Field,Gym,Coworking Space,Cuban Restaurant,Distribution Center
5,Central Toronto,1,Breakfast Spot,Hotel,Department Store,Dog Run,Food & Drink Shop,Park,Gym,Gluten-free Restaurant,Discount Store,Event Space
7,Central Toronto,1,Coffee Shop,Dessert Shop,Café,Sandwich Place,Italian Restaurant,Park,Indian Restaurant,Diner,Restaurant,Seafood Restaurant
9,Central Toronto,1,Light Rail Station,Coffee Shop,Supermarket,Liquor Store,Wings Joint,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
11,Downtown Toronto,1,Coffee Shop,Bakery,Pizza Place,Restaurant,Café,Pharmacy,Italian Restaurant,Park,Outdoor Sculpture,Ice Cream Shop
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Burger Joint,Café,Fast Food Restaurant,Grocery Store,Hotel
13,Downtown Toronto,1,Coffee Shop,Breakfast Spot,Restaurant,Bakery,Distribution Center,Mexican Restaurant,Pub,Electronics Store,Event Space,Spa


venues related with food, restaurants.

### Cluster 3

In [38]:
etoronto_merged.loc[etoronto_merged['Cluster Labels'] == 2, etoronto_merged.columns[[1] + list(range(5, etoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Spa,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


spa witn 2nd is a distrubution center, very different.